In [1]:
import os, sys
os.chdir('../')

In [2]:
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import loss as module_loss
import model.metric as module_metric
import model.model as module_arch

import easydict
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

import data_loader.data_loaders as module_data
import model.model as module_arch

from selection.svd_classifier import *
from selection.gmm import *
from selection.util import *

from utils.parse_config import ConfigParser
from utils.util import *
from utils.args import *

In [3]:
config_file = './hyperparams/multistep/config_cifar100_cce_rn34.json'
with open(config_file, 'r') as f:
    config = json.load(f)

# resume_path = './rn34/multistep_asym_40_elr.pth'

In [4]:
def decode(path):
    items = path.split('_')
    noisetype = True if items[2]=='asym' else False
    noiserate = float(items[3]) * 0.01
    
    return noisetype, noiserate, items[4].split('.')[0]

In [5]:
def make_parse(resume_path, config, noise_rate, noisetype):
    parse = easydict.EasyDict({
    "load_name" : resume_path,
    "reinit": False,
    "distill_mode": 'loss'
    })
    
    config['trainer']['percent'] = noise_rate
    config['trainer']['asym'] = noisetype
    
    return parse, config

In [6]:
def extract_cleanidx(teacher, data_loader, parse, print_statistics = True):
    teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    teacher = teacher.cuda()

    if not parse.reinit: teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    for params in teacher.parameters(): params.requires_grad = False
    
    if 'fine' in parse.distill_mode:
        features, labels = get_features(teacher, data_loader)
        clean_labels = fine(current_features=features, current_labels=labels, fit = parse.distill_mode)
    elif 'loss' in parse.distill_mode:
        clean_labels, labels = cleansing_loss(teacher, data_loader)
    else:
        raise NotImplemented 
    if print_statistics: 
        selected, precision, recall, specificity, accuracy = return_statistics(data_loader, clean_labels, datanum=len(labels))
    
    return selected, precision, recall, specificity, accuracy

In [7]:
def make_pd_list(root, config, log_filename):
    random.seed(config['seed'])
    torch.manual_seed(config['seed'])
    torch.cuda.manual_seed_all(config['seed'])
    torch.backends.cudnn.deterministic = True
    np.random.seed(config['seed'])
    
    # load checkpoint path
    pathlist = os.listdir(root)
    pathlist = [path for path in pathlist if 'c100' in path]
#     pathlist = [path for path in pathlist if ('.pth' in path) and ('eigen' not in path) and ('kmeans' not in path) and ('c100') not in path]
    
    # initialize model
    model = module_arch.resnet34(num_classes=100)
    
    # make pandas file
    logcolumns = ['noisetype', 'noiserate', 'lossfunction', 'selected', 'precision', 'recall', 'specificity', 'accuracy']
    log_pd = pd.DataFrame(np.zeros([len(pathlist), len(logcolumns)]), columns = logcolumns)
    
    # write pandas file
    noisetypelst = ['']
    for i in range(len(pathlist)):
        noisetype, noiserate, lossfunction = decode(pathlist[i])
        parse, config = make_parse('./rn34/' + pathlist[i], config, noiserate, noisetype)
        
        # load original dataloader
        data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size= 100,
        shuffle=False,
        validation_split=0.0,
        num_batches=config['data_loader']['args']['num_batches'],
        training=True,
        num_workers=config['data_loader']['args']['num_workers'],
        pin_memory=config['data_loader']['args']['pin_memory'],
        config=config)
        
        selected, precision, recall, specificity, accuracy = extract_cleanidx(model, data_loader, parse)
        log_pd.loc[i] = [str(noisetype), str(noiserate), lossfunction, selected, precision, recall, specificity, accuracy]
        log_pd.to_csv(log_filename)
        
    return log_pd

In [8]:
make_pd_list(root = './checkpoint/rn34/', config=config, log_filename = 'c100_loss_pretrained_statistics.csv')

Files already downloaded and verified
Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 147.33it/s]


Noisy: 9993, Clean: 40007
Selected samples: 39490 
Precision: 0.9473 
Recall: 0.935 
Specificity: 0.7917
Accuracy: 0.9064 
Fraction of clean samples/selected samples: 0.9473
Files already downloaded and verified
Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 146.66it/s]


Noisy: 9993, Clean: 40007
Selected samples: 40097 
Precision: 0.9176 
Recall: 0.9197 
Specificity: 0.6696
Accuracy: 0.8697 
Fraction of clean samples/selected samples: 0.9176
Files already downloaded and verified
Train: 50000 Val: 0


...

  0%|          | 0/500 [00:00<?, ?it/s]

Train: 50000 Val: 0


100%|██████████| 500/500 [00:03<00:00, 165.89it/s]

Noisy: 9905, Clean: 40095
Selected samples: 39616 
Precision: 0.9883 
Recall: 0.9765 
Specificity: 0.9532
Accuracy: 0.9719 
Fraction of clean samples/selected samples: 0.9883


,noisetype,noiserate,lossfunction,selected,precision,recall,specificity,accuracy
0,True,0.2,cce,39490.0,0.9473,0.9350,0.7917,0.9064
1,True,0.2,sce,40097.0,0.9176,0.9197,0.6696,0.8697
2,False,0.2,sce,48418.0,0.8261,0.9976,0.1498,0.8296
3,True,0.3,gce,37514.0,0.8889,0.9489,0.7196,0.8808
4,True,0.4,sce,37394.0,0.6437,0.7985,0.3290,0.6120
5,True,0.3,sce,37479.0,0.8188,0.8732,0.5428,0.7750
6,True,0.4,gce,37831.0,0.7210,0.9049,0.4685,0.7316
7,False,0.6,elr,23634.0,0.7678,0.8940,0.8152,0.8472
8,False,0.8,gce,18675.0,0.4848,0.8702,0.7570,0.7806
9,True,0.1,elr,41410.0,0.9974,0.9172,0.9787,0.9233
